<a href="https://colab.research.google.com/github/HyeonminNam/BOK4/blob/master/tone_mkt_analysis/200803_tone_analysis_with_other_dictionary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 다른 사전 활용해서 tone 측정하기

In [ ]:
import pandas as pd
from ekonlpy.sentiment import MPCK

# 문장의 polarity 판단하는 함수
def pol_score(sentence, hawkish_lst, dovish_lst):
    mpck = MPCK()
    token_lst = mpck.tokenize(sentence)
    ngram_lst = token_lst + mpck.ngramize(token_lst)
    hawkish = 0
    dovish = 0
    for x in ngram_lst:
        if x.lower() in hawkish_lst:
            hawkish += 1
        elif x.lower() in dovish_lst:
            dovish += 1
    # hawkish ngram과 dovish ngram의 수가 모두 0이거나 동일하면 polarity score 계산하지 않고 중립 문장으로 분류.
    if hawkish == dovish:
        return 'n'
    # pol_score가 0보다 크면 긍정, 0보다 작으면 부정, 0이면 중립인 문장
    pol_score = (hawkish-dovish)/(hawkish+dovish)
    if pol_score > 0:
        return 'h'
    elif pol_score < 0:
        return 'd'

In [ ]:
df_min = pd.read_csv('labeled_bok_minutes.csv')
lst = ['bond', 'minutes', 'news', 'ex_bond', 'ex_minutes', 'ex_news']

for x in lst:
    dic_df = pd.read_csv('polarity_dic_{}.csv'.format(x))
    min_pol = pd.DataFrame(columns = ['date', 'minutes', 'sentences', 'tone_mkt', 'call_rate', 'call_label'], index = df_min.index)
    
    # 의사록 자료의 데이터들 바탕으로 새로운 데이터프레임에 데이터 저장
    for idx, row in df_min.iterrows():
        min_pol['date'][idx] = row['date']
        min_pol['minutes'][idx] = row['minutes']
        min_pol['call_rate'][idx] = row['call_rate']
        min_pol['call_label'][idx] = row['call_label']
        # 의사록 내용 각 부분을  문장 단위로 나누고 다시 모두 합친 리스트 만들기
        sentence_lst = []
        if type(row['Economic Situation']) != float:
            sentence_lst += row['Economic Situation'].split('@@@')
        if type(row['Economic Situation']) != float:
            sentence_lst += row['Foreign Currency'].split('@@@')
        if type(row['Financial Markets']) != float:
            sentence_lst += row['Financial Markets'].split('@@@')
        min_pol['date'][idx] = row['date']
        min_pol['sentences'][idx] = sentence_lst
        
    hawkish_lst = list(dic_df[dic_df['label'] == 'hawkish']['tn'])
    dovish_lst = list(dic_df[dic_df['label'] == 'dovish']['tn'])
    
        # 문장의 polarity를 측정한 뒤에 이를 바탕으로 tone_mkt 계산하여 저장
    for idx, row in min_pol.iterrows():
        hawkish_tone = 0
        dovish_tone = 0
        for sentence in row['sentences']:
            pol_sentence = pol_score(sentence, hawkish_lst, dovish_lst)
            if pol_sentence == 'h':
                hawkish_tone += 1
            elif pol_sentence == 'd':
                dovish_tone += 1
        try:
            tone_mkt = (hawkish_tone - dovish_tone) / (hawkish_tone + dovish_tone)
        except:
            tone_mkt = 0
        min_pol['tone_mkt'][idx] = tone_mkt
        print('{}번 행 의사록 완료'.format(idx))
        
        # csv 파일로 저장
    min_pol.to_csv('minutes_tone_dic_{}.csv'.format(x), encoding='utf-8')
    print('{} 사전 활용해서 의사록 tone 분석 완료'.format(x))